In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/face-rec/data/45.jpg
/kaggle/input/face-rec/data/56.jpg
/kaggle/input/face-rec/data/20.jpg
/kaggle/input/face-rec/data/58.jpg
/kaggle/input/face-rec/data/6.jpg
/kaggle/input/face-rec/data/5.jpg
/kaggle/input/face-rec/data/8.jpg
/kaggle/input/face-rec/data/67.jpg
/kaggle/input/face-rec/data/30.jpg
/kaggle/input/face-rec/data/38.jpg
/kaggle/input/face-rec/data/42.jpg
/kaggle/input/face-rec/data/33.jpg
/kaggle/input/face-rec/data/10.jpg
/kaggle/input/face-rec/data/54.jpg
/kaggle/input/face-rec/data/0.jpg
/kaggle/input/face-rec/data/62.jpg
/kaggle/input/face-rec/data/35.jpg
/kaggle/input/face-rec/data/61.jpg
/kaggle/input/face-rec/data/59.jpg
/kaggle/input/face-rec/data/41.jpg
/kaggle/input/face-rec/data/60.jpg
/kaggle/input/face-rec/data/57.jpg
/kaggle/input/face-rec/data/9.jpg
/kaggle/input/face-rec/data/37.jpg
/kaggle/input/face-rec/data/1.jpg
/kaggle/input/face-rec/data/69.jpg
/kaggle/input/face-rec/data/46.jpg
/kaggle/input/face-rec/data/44.jpg
/kaggle/input/face-rec/dat

In [2]:
!pip install face-recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.4 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=b0ed332231d2ed6624ae7d8856341188b8c7c385fc0979313a93d91325c11eb9
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [3]:
!pip install insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1057641 sha256=74f5e3cbe5bac0e8c99e8875dc7ef8738d0cecafcb2ecf5e0c32a2169e1b264c
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [4]:
!pip install dlib -q

In [6]:
import face_recognition
import cv2
import os

# Paths
input_folder = "/kaggle/input/face-rec/data"         # Folder with raw images
output_folder = "/kaggle/working/faces"       # Folder to save cropped face images
os.makedirs(output_folder, exist_ok=True)

# Process each image
for idx, filename in enumerate(os.listdir(input_folder)):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        input_path = os.path.join(input_folder, filename)
        image = face_recognition.load_image_file(input_path)
        
        # Detect faces
        face_locations = face_recognition.face_locations(image)
        
        for face_idx, (top, right, bottom, left) in enumerate(face_locations):
            face_image = image[top:bottom, left:right]
            face_bgr = cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR)
            
            # Save face
            output_path = os.path.join(output_folder, f"{idx}_{face_idx}.jpg")
            cv2.imwrite(output_path, face_bgr)

print("Done extracting faces.")


Done extracting faces.


In [9]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.11.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.7 MB/s eta 0:00:00:00:0100:01


In [11]:
!pip install onnxruntime-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


In [13]:
import os
import cv2
import faiss
import numpy as np
from tqdm import tqdm
from insightface.app import FaceAnalysis

# ----------------- CONFIG -----------------
INPUT_FOLDER = '/kaggle/input/face-rec/data'        # Folder with full images
OUTPUT_FOLDER = '/kaggle/working/insight_face'      # Folder to save cropped face images (optional)
EMBEDDING_DIM = 512          # ArcFace output size
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ----------------- INIT MODEL -----------------
face_app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])  # or CPUExecutionProvider
face_app.prepare(ctx_id=0)

# ----------------- FAISS SETUP -----------------
index = faiss.IndexFlatL2(EMBEDDING_DIM)  # Use L2 distance (Euclidean)
embeddings = []
face_paths = []

# ----------------- PROCESS IMAGES -----------------
for img_name in tqdm(os.listdir(INPUT_FOLDER)):
    if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    
    img_path = os.path.join(INPUT_FOLDER, img_name)
    img = cv2.imread(img_path)
    faces = face_app.get(img)

    for i, face in enumerate(faces):
        # Save cropped face image
        x1, y1, x2, y2 = face.bbox.astype(int)
        face_crop = img[y1:y2, x1:x2]
        face_filename = f"{os.path.splitext(img_name)[0]}_{i}.jpg"
        save_path = os.path.join(OUTPUT_FOLDER, face_filename)
        cv2.imwrite(save_path, face_crop)

        # Store embedding
        embeddings.append(face.embedding)
        face_paths.append(face_filename)

# ----------------- BUILD FAISS INDEX -----------------
embeddings_np = np.array(embeddings).astype('float32')
index.add(embeddings_np)

# ----------------- SAVE METADATA -----------------
np.save("face_embeddings.npy", embeddings_np)
with open("face_paths.txt", "w") as f:
    for p in face_paths:
        f.write(f"{p}\n")

print(f"✅ Indexed {len(embeddings)} faces.")


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

  0%|          | 0/71 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
100%|██████████| 71/71 [00:03<00:00, 20.59it/s]

✅ Indexed 73 faces.


In [14]:
import faiss
import numpy as np

# Load your existing embeddings
embeddings = np.load("face_embeddings.npy").astype("float32")

# Create FAISS index (L2 distance)
index = faiss.IndexFlatL2(embeddings.shape[1])  # 512 for ArcFace
index.add(embeddings)

# Save the index to file
faiss.write_index(index, "faiss.index")

print(f"✅ FAISS index created and saved. Total vectors: {index.ntotal}")


✅ FAISS index created and saved. Total vectors: 73


In [20]:
import os
import cv2
import faiss
import numpy as np
from insightface.app import FaceAnalysis

# ----------------- CONFIG -----------------
QUERY_IMAGE = '/kaggle/input/face-rec/data/2.jpg'
FACE_IMAGES_DIR = '/kaggle/working/insight_face'
TOP_K = 5

# ----------------- LOAD MODEL -----------------
face_app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
face_app.prepare(ctx_id=0)

# ----------------- LOAD FAISS & METADATA -----------------
index = faiss.IndexFlatL2(512)
embeddings = np.load("face_embeddings.npy").astype("float32")
index.add(embeddings)

with open("face_paths.txt", "r") as f:
    face_paths = [line.strip() for line in f]

# ----------------- SAFELY LOAD QUERY IMAGE -----------------
if not os.path.exists(QUERY_IMAGE):
    raise FileNotFoundError(f"❌ QUERY_IMAGE not found: {QUERY_IMAGE}")

query_img = cv2.imread(QUERY_IMAGE)
if query_img is None:
    raise ValueError(f"❌ Unable to read image at: {QUERY_IMAGE}")

# ----------------- RUN FACE SEARCH -----------------
faces = face_app.get(query_img)
if not faces:
    print("❌ No face detected in the query image.")
else:
    query_embedding = np.expand_dims(faces[0].embedding.astype('float32'), axis=0)
    D, I = index.search(query_embedding, TOP_K)

    print("✅ Top matches:")
    for rank, idx in enumerate(I[0]):
        match_path = os.path.join(FACE_IMAGES_DIR, face_paths[idx])
        print(f"Rank {rank+1}: {match_path} (Distance: {D[0][rank]:.4f})")

        # Save result images
        match_img = cv2.imread(match_path)
        if match_img is not None:
            out_path = f"/kaggle/working/match_{rank+1}.jpg"
            cv2.imwrite(out_path, match_img)
        else:
            print(f"⚠️ Warning: Couldn't load match image at {match_path}")


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o